# Models stacking pipeline

In [1]:
import sys
sys.path.insert(0, '../src') 

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import json
from typing import Dict

from sklearn.model_selection import train_test_split

from sklearn.base import BaseEstimator
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

#from model import (
    #get_pm25_data_for_modelling,
    #split_df_for_ts_modelling_percentage,
    #split_df_for_ts_modelling_date_range
#)

## Get data

In [ ]:
def read_json(json_path: str) -> Dict:
    """
    Reads JSON file.
    :param json_path: full path to JSON file
    :return: Python dictionary
    """
    with open(json_path, 'r') as infile:
        return json.load(infile)

In [4]:
# Data files: https://github.com/ksatola/air-polution/tree/master/agh/data

def get_pm25_data_for_modelling(model_type: str = 'ml',
                                forecast_type: str = 'h') -> pd.DataFrame:
    """
    Reads HDF file with analytical view prepared for time series or machine learning modelling.
    :param model_type: 'ts' for time series analytical model or 'ml' for machine learning model
    :param forecast_type: 'd' for daily or 'h' for hourly data
    :return: pandas DataFrame
    """
    #config = read_json('../config/pm25_model.json')
    data_folder = 'data/'#config['data_folder']

    if model_type == 'ml':
        if forecast_type == 'h':
            data_file_hdf = data_folder + 'dfpm25_2008-2018_ml_24hours_lags.hdf'
        else:
            data_file_hdf = data_folder + 'dfpm25_2008-2018_ml_7days_lags.hdf'
    else:
        if forecast_type == 'h':
            data_file_hdf = data_folder + 'dfpm25_2008-2018_hourly.hdf'
        else:
            data_file_hdf = data_folder + 'dfpm25_2008-2018_daily.hdf'

    df = pd.read_hdf(path_or_buf=data_file_hdf, key="df")
    #logger.info(f'Dataframe loaded: {data_file_hdf}')
    #logger.info(f'Dataframe size: {df.shape}')
    print(f'Dataframe loaded: {data_file_hdf}')
    print(f'Dataframe size: {df.shape}')

    return df

In [5]:
dfd = get_pm25_data_for_modelling('ml', 'd')
dfd.head()

Dataframe loaded: data/dfpm25_2008-2018_ml_7days_lags.hdf
Dataframe size: (4015, 13)


,t,t-1,t-2,t-3,t-4,month,day,hour,dayofyear,weekofyear,dayofweek,quarter,season
0,57.312500,42.979167,46.104167,30.958333,53.586957,1,5,0,5,1,5,1,1
1,36.062500,57.312500,42.979167,46.104167,30.958333,1,6,0,6,1,6,1,1
2,46.083333,36.062500,57.312500,42.979167,46.104167,1,7,0,7,2,0,1,1
3,45.041667,46.083333,36.062500,57.312500,42.979167,1,8,0,8,2,1,1,1
4,101.375000,45.041667,46.083333,36.062500,57.312500,1,9,0,9,2,2,1,1


In [6]:
df = dfd.copy()
df.head()

,t,t-1,t-2,t-3,t-4,month,day,hour,dayofyear,weekofyear,dayofweek,quarter,season
0,57.312500,42.979167,46.104167,30.958333,53.586957,1,5,0,5,1,5,1,1
1,36.062500,57.312500,42.979167,46.104167,30.958333,1,6,0,6,1,6,1,1
2,46.083333,36.062500,57.312500,42.979167,46.104167,1,7,0,7,2,0,1,1
3,45.041667,46.083333,36.062500,57.312500,42.979167,1,8,0,8,2,1,1,1
4,101.375000,45.041667,46.083333,36.062500,57.312500,1,9,0,9,2,2,1,1


## Train test split

In [7]:
# Define first past/future cutoff point in time offset (1 year of data)
#cut_off_offset = 365*24 # for hourly data
cut_off_offset = 365 # for daily data

In [8]:
def split_df_for_ml_modelling_offset(data: pd.DataFrame,
                                     target_col: str = 't',
                                     cut_off_offset: int = 365) \
        -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):
    """

    :param data:
    :param target_col:
    :param cut_off_offset:
    :return:
    """

    # Take entire dataset and split it to train/test
    # according to train_test_split_position using cut_off_offset
    train_test_split_position = int(len(data) - cut_off_offset)

    X_train = data[0:train_test_split_position].copy()
    X_test = data[train_test_split_position:].copy()

    # Split datasets into independent variables dataset columns and dependent variable column
    y_train = X_train.pop(target_col)
    y_test = X_test.pop(target_col)

    return X_train, X_test, y_train, y_test

In [9]:
# For this method to work properly, observations in the data frame must be ordered by time (the greater index, the recent data)
# Train test split
X_train, X_test, y_train, y_test = split_df_for_ml_modelling_offset(data=df, target_col='t', cut_off_offset=cut_off_offset)

In [11]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3650, 12), (3650,), (365, 12), (365,))

## Super learner (stacking)

### Models initialization

In [16]:
best_models = [
    LinearRegression({'fit_intercept': True, 'n_jobs': -1, 'normalize': False}),
    ElasticNet(**{'alpha': 0.001, 'fit_intercept': True, 'l1_ratio': 0.2, 'max_iter': 10, 'selection': 'random'}),
    RandomForestRegressor(**{'n_jobs': -1, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 4, 'criterion': 'mae'}),
]

for model in best_models:
    print(type(model).__name__)

LinearRegression
ElasticNet
RandomForestRegressor


### Analytical view for the Super learner (based on predictions of basic models)

In [13]:
def get_analytical_view_for_meta_model(X_train: pd.DataFrame,
                                       y_train: pd.DataFrame,
                                       models: list,
                                       n_splits: int = 10
                                       ) -> (pd.DataFrame, pd.DataFrame):
    """
    Builds an analytical view based on kFold predictions of base models to be used by another
    model (out-of-fold predictions).
    :param X_train: input variables (pandas DataFrame)
    :param y_train: target variables (pandas DataFrame)
    :param models: list of base models
    :param n_splits: number of KFold splits
    :return: tuple of two pandas DataFrames, one with new input variables, and second with
    corresponding target variables (a copy of y_train)
    """

    # Define split of data
    kfold = KFold(n_splits=n_splits, shuffle=True)

    # Prepare empty data frame with column names (columns as models)
    columns = [type(model).__name__ for model in models]
    # and add target column
    columns = columns.append(y_train.columns[0])
    meta_X = pd.DataFrame(columns=columns)

    cv_fold_number = 1

    for train_indices, test_indices in kfold.split(X_train):
        #logger.debug(f'train: {train_indices}, len: {len(train_indices)}')
        #logger.debug(f'test: {test_indices}, len: {len(test_indices)}')

        #logger.info(f'CV fold number -> {cv_fold_number}')
        print(f'CV fold number -> {cv_fold_number}')
        cv_fold_number += 1

        # Get data
        train_X, test_X = X_train.iloc[train_indices], X_train.iloc[test_indices]
        train_y, test_y = y_train.iloc[train_indices], y_train.iloc[test_indices]
        #logger.debug(f'train_indices {train_indices.shape}')
        #logger.debug(f'test_indices {test_indices.shape}')

        # Add target variable
        fold_yhats = test_y.copy()

        # Fit and make predictions with each sub-model
        for model in models:
            model_name = type(model).__name__
            model.fit(train_X, train_y)
            yhat = model.predict(test_X)
            #logger.info(model_name)
            #logger.debug(f'train_X.shape {train_X.shape}')
            #logger.debug(f'train_y.shape {train_y.shape}')
            #logger.debug(f'yhat.shape {yhat.shape}')
            print(model_name)

            # Build fold-level results data frame, models as features
            fold_yhats[f'{model_name}'] = yhat

        #logger.debug(f'meta_X shape {meta_X.shape}')
        meta_X = pd.concat([meta_X, fold_yhats])

    # Take the target variable out from the dataset
    meta_y = meta_X.pop(y_train.columns[0]).to_frame()

    return meta_X, meta_y

In [14]:
y_train.to_frame()

,t
0,57.312500
1,36.062500
2,46.083333
3,45.041667
4,101.375000
...,...
3645,27.410467
3646,25.027338
3647,35.284304
3648,14.797779


In [18]:
%%time
meta_X, meta_y = get_analytical_view_for_meta_model(X_train, y_train.to_frame(), best_models, n_splits=10)

CV fold number -> 1
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 2
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 3
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 4
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 5
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 6
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 7
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 8
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 9
LinearRegression
ElasticNet
RandomForestRegressor
CV fold number -> 10
LinearRegression
ElasticNet
RandomForestRegressor
CPU times: user 8min 27s, sys: 2.11 s, total: 8min 29s
Wall time: 1min 15s


In [19]:
meta_X.head()

,LinearRegression,ElasticNet,RandomForestRegressor
3,51.233333,47.755285,77.988671
8,44.086754,44.017498,44.335413
12,32.631461,32.904881,43.637452
13,79.567028,73.996812,79.412572
23,25.361671,22.264138,16.254815


In [20]:
meta_y.head()

,t
3,45.041667
8,76.270833
12,72.916667
13,83.333333
23,25.687500


## Modelling

In [ ]:
def fit_base_models(X_train: pd.DataFrame,
                    y_train: pd.DataFrame,
                    models) -> list:
    """
    Fits all base models on the training dataset.
    :param X_train: input variables (pandas DataFrame)
    :param y_train: target variables (pandas DataFrame)
    :param models: list of base models
    :return: list of fitted sklearn models
    """
    for model in models:
        model.fit(X_train, y_train)

    return models

In [ ]:
%%time
# Fit base models on entire training dataset
models = fit_base_models(X_train, y_train, models)

In [ ]:
def fit_meta_model(X_train: pd.DataFrame,
                   y_train: pd.DataFrame,
                   meta_model: BaseEstimator = LinearRegression()) -> BaseEstimator:
    """
    Fits a meta model on the training dataset.
    :param X_train: input variables (pandas DataFrame)
    :param y_train: target variables (pandas DataFrame)
    :param meta_model: sklearn regression estimator class
    :return: fitted sklearn model
    """
    model = meta_model
    model.fit(X_train, y_train)
    return model

In [ ]:
%%time
# Fit the meta model (using sklearn.linear_model.LinearRegression)
meta_model = fit_meta_model(meta_X, meta_y)

In [ ]:
def get_rmse(observed: pd.Series, predicted: pd.Series) -> np.float:
    """
    Calculates RMSE - Root Mean Squared Error between two series
    :param observed: pandas series with observed values (the truth)
    :param predicted: pandas series with predicted values
    :return: RMSE value
    """
    mse = mean_squared_error(observed, predicted)
    rmse = round(np.sqrt(mse), 4)
    return rmse

In [ ]:
def evaluate_models(X_test: pd.DataFrame, y_test: pd.DataFrame, models: list) -> None:
    """
    Evaluates a list of models on a dataset using RMSE (Root Mean Squared Error).
    :param X_test: input variables (pandas DataFrame)
    :param y_test: target variables (pandas DataFrame)
    :param models: list of base models
    :return: None
    """
    for model in models:
        yhat = model.predict(X_test)
        #logger.info(f'{model.__class__.__name__} RMSE {get_rmse(y_test, yhat)}')
        print(f'{model.__class__.__name__} RMSE {get_rmse(y_test, yhat)}')

In [ ]:
# Evaluate the base models on the holdout (test) dataset
evaluate_models(X_test, y_test, models)

In [ ]:
def predict_with_super_learner(X_test: pd.DataFrame,
                               y_test: pd.DataFrame,
                               models: list,
                               meta_model: BaseEstimator) -> (pd.DataFrame, pd.DataFrame):
    """
    Makes predictions with stacked (meta) model.
    :param X_test: input variables (pandas DataFrame)
    :param y_test: target variables (pandas DataFrame)
    :param models: list of trained base models
    :param meta_model: sklearn model used for stacking
    :return: tuple of two pandas DataFrames, one with meta-model predictions, and second with
    corresponding target variables (a copy of y_test)
    """
    # yhats = y_test.copy()

    # Prepare empty data frame with column names (columns as models)
    columns = [type(model).__name__ for model in models]
    # and add target column
    columns = columns.append(y_test.columns[0])
    models_yhat = pd.DataFrame(columns=columns)

    # Add target variable
    yhats = y_test.copy()

    for model in models:
        model_name = type(model).__name__
        yhat = model.predict(X_test)
        #logger.info(model_name)
        print(model_name)
        #logger.debug(f'yhat.shape {yhat.shape}')

        # Build results data frame, models as features
        yhats[f'{model_name}'] = yhat

    models_yhat = pd.concat([models_yhat, yhats])
    # Take the target variable out from the dataset
    meta_target = models_yhat.pop(y_test.columns[0]).to_frame()

    # Predict
    meta_yhat = meta_model.predict(models_yhat)

    return meta_yhat, meta_target

## Predict with Super learner (on test data)

In [ ]:
# Use the super learner (base and meta models) to make predictions on the holdout dataset 
# and evaluate the performance of the approach
meta_yhat, meta_target = predict_with_super_learner(X_test, y_test.to_frame(), models, meta_model)
print(f'Super Learner RMSE {get_rmse(y_test, meta_yhat)}')